In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
matplotlib.style.use('ggplot')

import rows
import locale

print("Imported!")

Imported!


In [2]:
from pathlib import Path
prefeitos_path = Path('prefeitos-eleitos')

prefeitos = None

def get_state_name(filename):
    filename = str(filename)
    state_ext = filename.split(' - ')[-1:][0]
    state_ext = state_ext.split('/')[-1:][0]
    state = state_ext.split('.')[:-1][0]
    return state.capitalize()
    

with rows.locale_context(name='pt_BR.UTF-8', category=locale.LC_NUMERIC):
    for csv_filename in prefeitos_path.iterdir():
        if not csv_filename.is_file():
            continue
        with open(str(csv_filename), 'rb') as csv_file:
            if prefeitos is None:
                prefeitos = rows.import_from_csv(csv_file)
                prefeitos['state'] = [get_state_name(csv_filename)] * len(prefeitos)
            else:
                csv_tmp = rows.import_from_csv(csv_file)
                csv_tmp['state'] = [get_state_name(csv_filename)] * len(csv_tmp)
                for dic in rows.export_to_dicts(csv_tmp):
                    prefeitos.append(dic)

def segundo_turno(pref):
    return pref.split('* ')[-1:][0], bool(len(pref.split('* ')) - 1)

segundo_turno = [
    segundo_turno(prefeito)for prefeito in prefeitos['nome']
]
prefeitos['nome'] = [segundo[0] for segundo in segundo_turno]
prefeitos['segundo_turno'] = [segundo[1] for segundo in segundo_turno]

In [3]:
df = pd.DataFrame({field: prefeitos[field] for field in prefeitos.field_names})
df.head()

,municipio,nome,num,partidocoligacao,segundo_turno,state,validos,votacao
0,ABAÍRA,DIGA,14,PTB/PTB / PMDB / DEM / PTN / PRB,False,Bahia,0.513,2701
1,ABARÉ,FERNANDO TOLENTINO,13,PT/PT / PP / PHS / DEM / PTN / PR / PSL / PSD,False,Bahia,0.5108,5482
2,ACAJUTIBA,ALEX FREITAS,15,PMDB/PMDB / PSDB / PPL / SD / DEM,False,Bahia,0.5227,4482
3,ADUSTINA,PAULO SERGIO,17,PSL/PSL / PT / PTN / SD,False,Bahia,0.5186,4644
4,ÁGUA FRIA,MANOEL POTINHA,25,DEM/DEM / PDT / PMB / PV,False,Bahia,0.5113,4617


In [4]:
with open('consolidado-prefeitos.csv', 'wb') as f:
    f.write(rows.export_to_csv(prefeitos))


In [5]:
prefeitos.field_names

['municipio',
 'num',
 'nome',
 'partidocoligacao',
 'votacao',
 'validos',
 'state',
 'segundo_turno']

In [6]:
count_municipios = df.groupby(['state', 'municipio']).count()
with open('municipios-brasil.csv', 'w') as f:
    f.write(count_municipios.to_csv())
count_municipios

nome  num  partidocoligacao  \
state     municipio                                                   
Acre      ACRELÂNDIA                       1    1                 1   
          ASSIS BRASIL                     1    1                 1   
          BRASILÉIA                        1    1                 1   
          BUJARI                           1    1                 1   
          CAPIXABA                         1    1                 1   
          CRUZEIRO DO SUL                  1    1                 1   
          EPITACIOLÂNDIA                   1    1                 1   
          FEIJÓ                            1    1                 1   
          JORDÃO                           1    1                 1   
          MANOEL URBANO                    1    1                 1   
          MARECHAL THAUMATURGO             1    1                 1   
          MÂNCIO LIMA                      1    1                 1   
          PLÁCIDO DE CASTRO                1    1                 1   
          PORTO ACRE                       1    1                 1   
          PORTO WALTER                     1    1                 1   
          RIO BRANCO                       1    1                 1   
          RODRIGUES ALVES                  1    1                 1   
          SANTA ROSA DO PURUS              1    1                 1   
          SENA MADUREIRA                   1    1                 1   
          SENADOR GUIOMARD                 1    1                 1   
          TARAUACÁ                         1    1                 1   
          XAPURI                           1    1                 1   
Alagoas   ANADIA                           1    1                 1   
          ARAPIRACA                        1    1                 1   
          ATALAIA                          1    1                 1   
          BARRA DE SANTO ANTÔNIO           1    1                 1   
          BARRA DE SÃO MIGUEL              1    1                 1   
          BATALHA                          1    1                 1   
          BELO MONTE                       1    1                 1   
          BELÉM                            1    1                 1   
...                                      ...  ...               ...   
Tocantins RIACHINHO                        1    1                 1   
          RIO DA CONCEIÇÃO                 1    1                 1   
          RIO DOS BOIS                     1    1                 1   
          RIO SONO                         1    1                 1   
          SAMPAIO                          1    1                 1   
          SANDOLÂNDIA                      1    1                 1   
          SANTA FÉ DO ARAGUAIA             1    1                 1   
          SANTA MARIA DO TOCANTINS         1    1                 1   
          SANTA RITA DO TOCANTINS          1    1                 1   
          SANTA ROSA DO TOCANTINS          1    1                 1   
          SANTA TEREZA DO TOCANTINS        1    1                 1   
          SANTA TEREZINHA DO TOCANTINS     1    1                 1   
          SILVANÓPOLIS                     1    1                 1   
          SUCUPIRA                         1    1                 1   
          SÃO BENTO DO TOCANTINS           1    1                 1   
          SÃO FÉLIX DO TOCANTINS           1    1                 1   
          SÃO MIGUEL DO TOCANTINS          1    1                 1   
          SÃO SALVADOR DO TOCANTINS        1    1                 1   
          SÃO SEBASTIÃO DO TOCANTINS       1    1                 1   
          SÃO VALÉRIO DA NATIVIDADE        1    1                 1   
          SÍTIO NOVO DO TOCANTINS          1    1                 1   
          TAGUATINGA                       1    1                 1   
          TAIPAS DO TOCANTINS              1    1                 1   
          TALISMÃ                          1    1               

In [7]:
with open('municipios-brasil.html', 'w') as f:
    f.write(count_municipios.to_html())

In [8]:
plt.close('all')

In [9]:
segundo_turno = df[df.segundo_turno == True]
group_segundo = segundo_turno.groupby('municipio')
segundo_turno.cumsum()
segundo_turno.plot.barh()